## 图像灰度测试

In [1]:
import numpy as np
import cv2

In [2]:
def imshow(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
rabbit_img = '.\\img\\rabbit1.jpg'

In [4]:
rabbit = cv2.imread(rabbit_img)
rabbit = cv2.resize(rabbit, (480, 270))
print(rabbit.shape)
imshow('rabbit', rabbit)

(270, 480, 3)


In [5]:
rabbit_cvt = np.dot(rabbit, [0.114, 0.587, 0.299])
# 必须强转为uin8类型
imshow('rabbit_gray', (rabbit_cvt+0.5).astype(np.uint8))

In [6]:
def cvtColor2Gray(img):
    gray = (np.dot(img, np.array([0.114, 0.587, 0.299]))+0.5).astype(np.uint8)
    return gray

# cvtColor2Gray(rabbit)

## Triangle阈值原理

In [7]:
def Threshold_Triangle(img):
    """
    图片置灰处理
    """
    gray = cvtColor2Gray(img)
    gray_shape = gray.shape # 记录置灰后图片数据shape,然后将图片数据展平
    gray = gray.flatten()

    histogram = np.zeros((256))
    for i in range(len(gray)):
        histogram[gray[i]] += 1
    
    for i in range(len(histogram)):
        if histogram[i] != 0:
            bleft = i-1
            break

    for i in range(len(histogram)-1, -1, -1):
        if histogram[i] != 0:
            bright = i+1
            break

    flipped = False
    max_id = np.argmax(histogram)
    max_pixel = histogram[max_id]
    if max_id-bleft < bright-max_id:
        flipped = True
        start = 0
        end = 255-start
        while start < end:
            temp = histogram[start]
            histogram[start] = histogram[end]
            histogram[end] = temp
            start += 1
            end -= 1
        bleft = 255-bright
        max_id = 255-max_id

    """
    直线一般方程:
    A = y2-y1, B = x1-x2, C = x2*y1 - x1*y2
    点到直线距离:
    d = |A*x + B*y + C|/(A**2 + B**2)**0.5
    点到直线最大值： A*x + B*y最大
    """
    a = max_pixel-histogram[bleft] # A = y2-y1
    b = bleft-max_id               # B = x1-x2, C = x2*y1 - x1*y2
    maxdist = 0
    for i in range(bleft+1, max_id+1):
        dist = a*i + b*histogram[i]
        if dist > maxdist:
            maxdist = dist
            threshold = i
    threshold -= 1

    if flipped:
        threshold = 255-threshold

    for i in range(len(gray)):
        if gray[i] > threshold:
            gray[i] = 255
        else:
            gray[i] = 0
    return np.reshape(gray, gray_shape) # 将图片数据还原为原shape

In [8]:
threshold_triangle = Threshold_Triangle(rabbit)
imshow('threshold_triangle', threshold_triangle)

## OTSU阈值原理

In [9]:
def Threshold_OTSU(img):
    """
    图片置灰处理
    """
    gray = cvtColor2Gray(img)
    gray_shape = gray.shape  # 记录置灰后图片数据shape,然后将图片数据展平
    gray = gray.flatten()

    histogram = np.zeros((256))
    valid_pixel_sum = 0
    for i in range(len(gray)):
        histogram[gray[i]] += 1
        if gray[i] != 0:
            valid_pixel_sum += 1
    
    for i in range(len(histogram)):
        if histogram[i] != 0:
            bleft = i
            break
    
    for i in range(len(histogram)-1, -1, -1):
        if histogram[i] != 0:
            bright = i
            break

    """
    假设:有阈值T,根据像素值分类为C1(pixel<T),C2(pixel>=T),
    * 像素分在C1,C2类概率为p0,p1,
    * C1,C2均值为avg0,avg1,图像均值为avgG
    >>> avg0*p0 + avg1*p1 = avgG                         (1)
    >>> p0+p1 = 1                                        (2)
    类间协方差为:
    >>> sigma**2 = p0*(avg0-avgG)**2 + p1*(avg1-avgG)**2 (3)
    >>> sigma**2 = p0*p1(avg0-avg1)**2                   (4)
    """
    max_variance = 0
    threshold = 0
    for i in range(bleft, bright+1):
        avg0, p0, avg1, p1 = (0, 0, 0, 0)
        for j in range(0, i+1):
            avg0 += histogram[j]
            p0 += j*histogram[j]
        if avg0 == 0:
            break
        p0 = p0/avg0
        avg0 = avg0/valid_pixel_sum

        for j in range(i+1, 255):
            avg1 += histogram[j]
            p1 += j*histogram[j]
        if avg1 == 0:
            break
        p1 = p1/avg1
        avg1 = avg1/valid_pixel_sum

        variance = avg0*avg1*(p1-p0)**2
        if variance > max_variance:
            max_variance = variance
            threshold = i

    for i in range(len(gray)):
        if gray[i] > threshold:
            gray[i] = 255
        else:
            gray[i] = 0
    return np.reshape(gray, gray_shape) # 将图片数据还原为原shape

In [10]:
threshold_otsu = Threshold_OTSU(rabbit)
imshow('threshold_otsu', threshold_otsu)